In [19]:
import import_hack
import steps
from data_store import sqlite_table_datastore as sqlds
from data_store import file_system_directory_datastore as fsds
from data_store import stream_ndarray_adapter_datastore as snads
from data_store import numpy_datastore as npds
from transformer import transformers as trs
from search import inverted_multi_index_searcher as imis
from quantization import pq_quantizer
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
img_dir_path = r'C:\data\images\brodatz\data.brodatz\size_213x213'
sqlite_db_path='example_find_nearest_by_histograms_with_multi_index'
sqlite_table_name='histograms'
n_subquantizers=2
n_clusters=256
sqltie_clusters_table_name='clusters_pq_{0}_{1}'.format(n_subquantizers, n_clusters)

In [3]:
image_bytes_ds = fsds.FileSystemDirectoryDataStore(dir_path=img_dir_path)
histograms_ds=sqlds.SQLiteTableDataStore(sqlite_db_path, sqlite_table_name)
image_bytes_ds.get_count()

999

In [4]:
transformers_=[trs.BytesToNdarray(), trs.NdarrayToOpencvMatrix(), trs.OpencvMatrixToHistogram(graycolor=True)]
steps.transform_step(image_bytes_ds, transformers_, histograms_ds)

In [5]:
histograms_ds_ndarrayds = snads.StreamNdarrayAdapterDataStore(histograms_ds)
print(histograms_ds_ndarrayds.get_items_sorted_by_ids().shape)
np_neighbor_ids_ds=npds.NumpyDataStore()

(999, 256, 1)


In [6]:
x=histograms_ds_ndarrayds.get_items_sorted_by_ids()
x_ids=histograms_ds_ndarrayds.get_ids_sorted()

In [7]:
pq_=pq_quantizer.PQQuantizer(n_clusters, n_subquantizers)
#be careful with pq: multiindex will consume at least (n_clusters^m_subquantizers)*4 bytes
cluster_centers_ds=sqlds.SQLiteTableDataStore(sqlite_db_path, sqltie_clusters_table_name)
steps.quantize_step(histograms_ds_ndarrayds, pq_, cluster_centers_ds)

In [8]:
cluster_centers_ndarray_ds=snads.StreamNdarrayAdapterDataStore(cluster_centers_ds)
cluster_centers=cluster_centers_ndarray_ds.get_items_sorted_by_ids()

In [9]:
cluster_centers.shape

(2, 256, 128)

In [14]:
searcher_=imis.InvertedMultiIndexSearcher(x, x_ids, cluster_centers)

In [15]:
np_neighbor_ids_ds=npds.NumpyDataStore()
steps.search_step(histograms_ds, searcher_, 20 , np_neighbor_ids_ds)

In [16]:
np_neighbor_ids_ds.get_items_sorted_by_ids()

array([[  1,   3,   8, ..., 477, 755, 754],
       [  2,   9,   6, ...,  96,  93, 995],
       [  3,   1,   2, ..., 995,  99,  96],
       ..., 
       [998, 997, 994, ..., 999,  45,  44],
       [998, 997, 994, ..., 475,  45,  44],
       [999, 992, 996, ...,  13, 125, 122]])

In [17]:
ids=histograms_ds_ndarrayds.get_ids_sorted()
source_ids=image_bytes_ds.get_ids_sorted()
np_neighbor_source_ids=npds.NumpyDataStore()
steps.transform_step(np_neighbor_ids_ds, [trs.TranslateByKeysTransformer(ids, source_ids)], np_neighbor_source_ids)

In [18]:
np_neighbor_source_ids.get_items_sorted_by_ids()

array([['D001_01.png', 'D001_03.png', 'D001_08.png', ..., 'D054_09.png',
        'D085_08.png', 'D085_07.png'],
       ['D001_02.png', 'D001_09.png', 'D001_06.png', ..., 'D011_06.png',
        'D011_03.png', 'D112_05.png'],
       ['D001_03.png', 'D001_01.png', 'D001_02.png', ..., 'D112_05.png',
        'D011_09.png', 'D011_06.png'],
       ..., 
       ['D112_08.png', 'D112_07.png', 'D112_04.png', ..., 'D112_09.png',
        'D005_09.png', 'D005_08.png'],
       ['D112_08.png', 'D112_07.png', 'D112_04.png', ..., 'D054_07.png',
        'D005_09.png', 'D005_08.png'],
       ['D112_09.png', 'D112_02.png', 'D112_06.png', ..., 'D002_04.png',
        'D015_08.png', 'D015_05.png']], dtype=object)

In [ ]:
import os
os.remove(sqlite_db_path)